In [9]:
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy

# pd.set_option('display.max_colwidth', 200)

# load spaCy model
# nlp = spacy.load("en_core_web_sm")

In [20]:
import spacy
from nltk import Tree

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

def tok_format(tok):
    return "_".join([tok.orth_, tok.tag_, tok.dep_])

def to_nltk_formatted_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(tok_format(node), [to_nltk_formatted_tree(child) for child in node.children])
    else:
        return tok_format(node)

In [21]:
en_nlp = spacy.load('en_core_web_sm')

doc = en_nlp("APT19 sent spearphishing emails with malicious attachments in RTF and XLSM formats to deliver initial exploits")

# displacy.render(doc, style='dep',jupyter=True)
tree = [to_nltk_formatted_tree(sent.root) for sent in doc.sents]
tree[0].draw()

[to_nltk_formatted_tree(sent.root).pretty_print() for sent in doc.sents]

                                                                   sent_VBD_ROOT                                                                             
        _________________________________________________________________|_________________________________________________________                           
       |                                          emails_NNS_dobj                                                                  |                         
       |                _________________________________|_______________                                                          |                          
       |               |                                            with_IN_prep                                                   |                         
       |               |                                                 |                                                         |                          
       |               |                         

[None]